## Case Study
# Quantifying fluorescent intensity images

We will work at some images of Bacillus subtilis, a gram-positive bacterium famous for its ability to enter a form of 'suspended animation' known as sporulation when environmental conditions get rough. In these images, all cells have been engineered to express Cyan Fluorescent Protein (CFP) once they enter a particular genetic state known as comptency.

These cells have been imaged under phase contrast and epifluorescence microscopy. These images were acquired by Caltech graduate student (and 2016 bootcamp TA) Griffin Chure.

Many of your image processing functions should come from scipy.ndimage and/or scikit-image.

Let's import the packages

In [ ]:
import numpy as np

# Our image processing tools
import skimage.filters
import skimage.io
import skimage.morphology
import skimage.exposure
import skimage.segmentation

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}


Let's start by just getting a look at the images to see what we're dealing with here. A collection of images can be found in imgs/data which will be used for this lesson and the following practice section. The specific images we will be looking at here will be noLac_phase_0000.tif and noLac_FITC_0000.tif.

In [ ]:
# Load the images
im_phase = skimage.io.imread('imgs/data/noLac_phase_0004.tif')
im_fl = skimage.io.imread('imgs/data/noLac_FITC_0004.tif')

# Display side-by-side
plt.subplot(1,2,1)
plt.imshow(im_phase,cmap=plt.cm.viridis)
plt.subplot(1,2,2)
plt.imshow(im_fl,cmap=plt.cm.viridis)
plt.show()

What we see:

1. that the illumination in the phase contrast image is not uniform and is darker at the top than on the bottom. 
2. a couple rogue pixels are ruining viewing the fluorescent image. 

So let's do a median filter on it to clean.

In [ ]:
# Structuring element
selem = skimage.morphology.square(3)

# Perform median filter
im_phase_filt = skimage.filters.median(im_phase, selem)
im_fl_filt = skimage.filters.median(im_fl, selem)

plt.subplot(1,2,1)
plt.imshow(im_phase_filt,cmap=plt.cm.viridis)
plt.subplot(1,2,2)
plt.imshow(im_fl_filt,cmap=plt.cm.viridis)
plt.show()

Let's now correct our illumination issues in the phase contrast image. 
We can correct for this non-uniform illumination by performing a Gaussian background subtraction. 

In [ ]:
# Apply a gaussian blur with a 50 pixel radius. 
im_phase_gauss = skimage.filters.gaussian(im_phase_filt, 50.0)

# Convert the median-filtered phase image to a float64
im_phase_float = skimage.img_as_float(im_phase_filt)

# Subtract our gaussian blurred image from the original.
im_phase_sub = im_phase_float - im_phase_gauss

plt.subplot(1,2,1)
plt.imshow(im_phase_filt,cmap=plt.cm.viridis)
plt.subplot(1,2,2)
plt.imshow(im_phase_sub,cmap=plt.cm.viridis)
plt.show()

Inspecting the images....

In [ ]:
# Indices of subimage
slc = np.s_[0:450, 50:500]
plt.imshow(im_phase_sub[slc], cmap=plt.cm.gray)
plt.show()

The image is littered with small dots. These are the result of condensation of water vapor on the glass in front of the CCD of the camera. Let's denoise the image using the total variation filter.  

In [ ]:
# Denoising the image
im_phase_tv = skimage.restoration.denoise_tv_chambolle(im_phase_sub, weight=0.005)
slc = np.s_[0:450, 50:500]
plt.imshow(im_phase_tv[slc], cmap=plt.cm.gray)
plt.show()

Assuming we have taken care of any illumination issues let's apply a global thresholding method on the phase images to separate bacteria from background.

In [ ]:
# Compute Otsu threshold value for median filtered image
thresh_otsu = skimage.filters.threshold_otsu(im_phase_tv)

# Construct thresholded image
im_bw = im_phase_tv < thresh_otsu

plt.subplot(1,2,1)
plt.imshow(im_phase_tv,cmap=plt.cm.viridis)
plt.subplot(1,2,2)
plt.imshow(im_bw,cmap=plt.cm.viridis)
plt.show()

Since we will be quantifying fluorescent intensity in individual bacteria, we want to make sure only whole bacteria are considered. Therefore, we should clear off any bacteria that are touching the border of the image, connected regions, remove small objects and then count. 

In [ ]:
# Clear border with 5 pixel buffer
im_bw = skimage.segmentation.clear_border(im_bw, buffer_size=5)

# Label binary image; background kwarg says value in im_bw to be background
im_labeled, n_labels = skimage.measure.label(im_bw, background=0, return_num=True)

# Extract region props
im_props = skimage.measure.regionprops(im_labeled, intensity_image=im_fl_filt)

# Make a filtered black and white image
im_bw_filt = im_labeled > 0

# Define cutoff size
cutoff = 150

# Loop through image properties and delete small objects
n_regions = 0
for prop in im_props:
    if prop.area < cutoff:
        im_bw_filt[im_labeled==prop.label] = 0
    else:
        n_regions += 1

plt.imshow(im_bw_filt, cmap=plt.cm.rainbow)
plt.show()

# Show number of regions
print('Number of individual regions = ', n_regions)

# Exercise

Create your own segmentation function to segment any phase contrast image of bacteria and count the number os cells. Try to implement: 

1. Correct for hot or bad pixels in an image.
2. Correct for uneven illumination.
3. Perform a thresholding operation.
4. Remove bacteria or objects near/touching the image border.
5. Remove objects that are too large (or too small) to be bacteria. 
6. Remove improperly segmented cells.
7. Return a labeled segmentation mask.
8. Count the number of objects. 

In [ ]:
def mysegmentation(im, image_mode='phase'):

    # Apply a median filter to remove hot pixels.

   
    # Perform gaussian subtraction
    im_sub = bg_subtract(im_filt)
    
    # Determine the thresholding method.


    # Determine the image mode and apply threshold.

       
    # Label the objects.


    # Apply the area and eccentricity bounds. 
    im_filt = area_ecc_filter(im_label)
    
    # Remove objects touching the border.

    
    # Relabel the image.

        
    return im_label

def bg_subtract(im):
    
    # Apply the gaussian filter.
    
    # Ensure the original image is a float
    
    # Subtract

    
    return  im_sub


def area_ecc_filter(im):
    
    # Extract the region props of the objects. 

    # Extract the areas and labels.

    # Make an empty image to add the approved cells.

    
    # Threshold the objects based on area and eccentricity
    area_bounds=(0,1e7)
    ecc_bounds=(0, 1)
    
    # Relabel the image.

    return im_filt

def count_cells(seg):
    # Extract region props
 

    # Make a filtered black and white image


    # Define cutoff size


    # Loop through image properties and delete small objects
            
    return n_regions

In [ ]:
ip_dist = 0.0636  # in units of µm per pixel.
area_bounds = (1/ip_dist**2, 10.0/ip_dist**2)
ecc_bounds = (0.8, 1.0)  # they are certainly not spheres. 

ecoli = skimage.io.imread('imgs/data/noLac_phase_0004.tif')
bsub_phase = skimage.io.imread('imgs/bsub_100x_phase.tif')
bsub_fluo = skimage.io.imread('imgs/bsub_100x_cfp.tif')

# Pass all images through our function.
ecoli_seg = mysegmentation(ecoli, image_mode='phase')
bsub_phase_seg = mysegmentation(bsub_phase, image_mode='phase')
bsub_fluo_seg = mysegmentation(bsub_fluo, image_mode='fluorescence')

In [ ]:
# Show number of regions
n_regions = count_cells(ecoli_seg)
print('Number of individual regions = ', n_regions)

n_regions = count_cells(bsub_phase_seg)
print('Number of individual regions = ', n_regions)

n_regions = count_cells(bsub_fluo_seg)
print('Number of individual regions = ', n_regions)